# Asuntojen hinnan muodostuminen

Tarkastellaan asuntojen hintojen muodustumista.

Datassa seuraavat muuttujat 
['Kaupunginosa','Huoneet','Talotiedot','m2','Vh','Neliohinta','Rv']

A. Millaisia korrelaatiota eri tekijöiden välillä on?
- korrelaatiomatriisi ja -kertoimet

B. Pääkomponenttianalyysi (PCA)
-> Olennaisesti neljä pääkomponenttia

Mitkä tekijät selittävät asunnon velatonta hintaa tai neliöhintaa? 
- Ennustava malli esimerkiksi: Vh ~ Kaupunginosa, Huoneet, Talotiedot, m2, Rv
- Ennustava malli esimerkiksi: Neliohinta ~ Kaupunginosa, Huoneet, Talotiedoet, m2, Rv
- Vh ~ m2, Kaupunginosa, Talotiedot, Rv

C. OLS 


In [337]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [338]:
rawdata='C://Python34/datasets/asunnot_250316_cleaned_numerical3.csv'
df_raw = pd.read_csv(rawdata,header=None)
column_names = ['Kaupunginosa','Huoneet','Talotiedot','m2','Vh','Neliohinta','Rv']
#print(df_raw.head(2))
df=df_raw.ix[:, [0,1,2,3,4,5,6]]




Select variables and target for linear regression

In [339]:
# Here chosen Talotiedot,m2,Rv
df_ols=df_raw.ix[:, [2,3,6]]
df_target=df_raw.ix[:,4]

In [340]:
# Full covariance table
covtable=np.cov(df,rowvar=False)
print(np.cov(df,rowvar=False))

[[  1.02510241e+01   1.96432279e-01   6.25219443e-01   1.24690533e+01
   -3.52271026e+03  -3.33294082e+02  -4.91624351e+00]
 [  1.96432279e-01   8.99178858e-01   4.80773950e-01   3.17630996e+01
    5.77472821e+04  -1.35255158e+02   3.72024540e-01]
 [  6.25219443e-01   4.80773950e-01   6.40349221e-01   2.33795224e+01
    4.46439027e+04  -6.14263143e+01  -2.31941482e+00]
 [  1.24690533e+01   3.17630996e+01   2.33795224e+01   1.80333955e+03
    2.76126320e+06  -9.35927400e+03  -5.74516527e+01]
 [ -3.52271026e+03   5.77472821e+04   4.46439027e+04   2.76126320e+06
    7.62388314e+09   1.97673039e+07   3.75706475e+05]
 [ -3.33294082e+02  -1.35255158e+02  -6.14263143e+01  -9.35927400e+03
    1.97673039e+07   5.27443598e+05   4.85604040e+03]
 [ -4.91624351e+00   3.72024540e-01  -2.31941482e+00  -5.74516527e+01
    3.75706475e+05   4.85604040e+03   3.21817197e+02]]


In [341]:
# Covariance table for linear regression
covtable_ols=np.cov(df_ols,rowvar=False)
print(np.cov(df_ols,rowvar=False))

[[  6.40349221e-01   2.33795224e+01  -2.31941482e+00]
 [  2.33795224e+01   1.80333955e+03  -5.74516527e+01]
 [ -2.31941482e+00  -5.74516527e+01   3.21817197e+02]]


In [342]:
covmatdim=min(df_ols.shape)
print(covmatdim) # kovarianssimatriisin dimensio


3


In [343]:
# Correlation coefficients for the variables chosen for linreg 
for i in range(covmatdim+1):
    for j in range(i+1,covmatdim):
        print('i,j,corrcoeff:',i,j,covtable_ols[i,j]/np.sqrt(covtable_ols[i,i]*covtable_ols[j,j]))


i,j,corrcoeff: 0 1 0.688000000094
i,j,corrcoeff: 0 2 -0.161571724668
i,j,corrcoeff: 1 2 -0.0754152453363


Note: As could be expected, there is correlation between Talotiedot and m2.

## A. Korreloivat tekijät

A. Mitkä tekijät korreloivat eniten keskenään? Kovarianssimatriisista:
Huoneet vs. Vh 10+4, 2+5
Talotiedot vs. Vh 10+4, 3+5
m2 vs. Vh 10+6, 4+5
Vh vs. neliohinta 10+5, 5+6 
Vh vs. Rv 10+3, 5+7
Rv vs. Neliohinta 10+3, 6+7   
Kaup. osa vs. Vh 10+3, 1+5

Korrelaatiokertoimia

In [344]:
#covtable.shape
print('Korrelaatiokertoimia')
print('huoneidan maara vs Vh:',covtable[1,4]/np.sqrt(covtable[1,1]*covtable[4,4]))
print('talot. vs Vh:',covtable[2,4]/np.sqrt(covtable[2,2]*covtable[4,4]))
print('m2 vs Vh:',covtable[3,4]/np.sqrt(covtable[3,3]*covtable[4,4]))
print('nelioh. vs Vh:',covtable[4,5]/np.sqrt(covtable[5,5]*covtable[4,4]))
print('Rv vs Vh:',covtable[6,4]/np.sqrt(covtable[4,4]*covtable[6,6]))
print('Rv vs nelioh:',covtable[6,5]/np.sqrt(covtable[5,5]*covtable[6,6]))
print('Kaup.osa vs Vh:',covtable[0,4]/np.sqrt(covtable[0,0]*covtable[4,4]))


Korrelaatiokertoimia
huoneidan maara vs Vh: 0.697461741747
talot. vs Vh: 0.638948117925
m2 vs Vh: 0.74469928379
nelioh. vs Vh: 0.311724769119
Rv vs Vh: 0.239858957082
Rv vs nelioh: 0.372725970525
Kaup.osa vs Vh: -0.0126010049556


## B. PCA

In [345]:
from sklearn.decomposition import PCA
pca = PCA()

In [346]:
df_pca = pca.fit_transform(df)
print(np.cov(df_pca, rowvar=False))
#print(df_pca)


[[  7.62393541e+09   2.88853279e-09  -6.95413641e-11   2.41785668e-12
   -3.57641301e-12   3.12446411e-12   1.01023873e-12]
 [  2.88853279e-09   4.76792956e+05  -4.84830637e-13   1.24250968e-12
   -1.59861184e-13   4.44253181e-14  -8.01054949e-14]
 [ -6.95413641e-11  -4.84830637e-13   3.13189438e+02   3.12638804e-14
    3.81370149e-15  -2.01206573e-16   8.40011821e-16]
 [  2.41785668e-12   1.24250968e-12   3.12638804e-14   1.88442141e+02
   -1.09314267e-17   1.70530257e-15   2.41584530e-15]
 [ -3.57641301e-12  -1.59861184e-13   3.81370149e-15  -1.09314267e-17
    1.00173991e+01  -1.09314267e-17  -7.68615940e-18]
 [  3.12446411e-12   4.44253181e-14  -2.01206573e-16   1.70530257e-15
   -1.09314267e-17   3.06190676e-01   0.00000000e+00]
 [  2.44304269e-12  -7.80066610e-14   7.58026121e-16   2.43770815e-15
   -7.68615940e-18   0.00000000e+00   2.32925497e-01]]


In [347]:
print(pca.explained_variance_)

[  7.60054911e+09   4.75330401e+05   3.12228735e+02   1.87864098e+02
   9.98667084e+00   3.05251441e-01   2.32211002e-01]


-> Four main factors explaining the variance

In [348]:

plt.plot(np.log(pca.explained_variance_),'bo');
plt.plot(np.log(pca.explained_variance_),'k');
plt.ylabel('log')
plt.show()

## C. OLS

In [349]:
xx= sm.add_constant(df_ols,prepend=False)
print(xx.head())
print(df_target.head())

     2     3       6  const
0  0.0  32.0  2016.0      1
1  0.0  35.0  1984.0      1
2  0.0  35.0  1975.0      1
3  0.0  45.0  2007.0      1
4  0.0  21.0  1960.0      1
0    135950.0
1     95000.0
2     90000.0
3    148500.0
4    114000.0
Name: 4, dtype: float64


In [350]:
results = sm.OLS(df_target, xx).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      4   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     241.1
Date:                Sat, 04 Jun 2016   Prob (F-statistic):           5.44e-82
Time:                        23:02:39   Log-Likelihood:                -3979.1
No. Observations:                 326   AIC:                             7966.
Df Residuals:                     322   BIC:                             7981.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
2           3.377e+04   4704.525      7.178      0.0